In [ ]:
clf = CountVectorizer()
X_train_cv =  clf.fit_transform(X_train)
X_test_cv = clf.transform(X_test)
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
X_train_tf = tf_transformer.transform(X_train_cv)
X_test_tf = tf_transformer.transform(X_test_cv)
nb_clf = MultinomialNB()
MultinomialNB()
nb_clf.fit(X_train_tf, y_train)
nb_pred = nb_clf.predict(X_test_tf)

In [ ]:
# Define LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
#  Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Convert the labels to one-hot encoding
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
# Train the model
from keras.callbacks import EarlyStopping
history = model.fit(X_train, y_train, epochs=10, batch_size=1024 , validation_split=0.1,
                   verbose=1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Test Accuracy: {:.4f}".format(accuracy))

In [ ]:
# Define important libraries for CNN Model
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
# Define the CNN Model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Convert the labels to one-hot encoding
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
# Train the model
from keras.callbacks import EarlyStopping
history = model.fit(X_train, y_train, epochs=10, batch_size=1024 , validation_split=0.1,
                   verbose=1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Test Accuracy: {:.4f}".format(accuracy))

In [ ]:
import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import DistilBertTokenizerFast



MAX_LEN = 128
# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the sentences and convert them to numerical input
input_ids = []
attention_masks = []

# For every sentence in the dataset
for sent in data['review']:
    # Encode the sentence using the tokenizer
    encoded_sent = tokenizer.encode_plus(
        text=sent,  # Sentence to encode
        add_special_tokens = True,  # Add [CLS] and [SEP]
        max_length = MAX_LEN,  # Truncate longer sentences
        pad_to_max_length = True,  # Pad shorter sentences
        return_attention_mask = True,  # Generate attention mask
        return_tensors='pt'  # Return PyTorch tensors
    )
    
    # Add the numerical input and attention mask to the lists
    input_ids.append(encoded_sent['input_ids'])
    attention_masks.append(encoded_sent['attention_mask'])
    
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,  # Number of output labels
    output_attentions=False,
    output_hidden_states=False,
)

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs = 4,
    per_device_train_batch_size = 128,
    per_device_eval_batch_size = 128,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_total_limit=1,
    learning_rate=2e-5,
)

# Assuming you have defined model, eval_dataset, and args
def compute_metrics(eval_prediction_output, labels=None):
    logits = eval_prediction_output.predictions
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(train_inputs, train_masks, train_labels),
    eval_dataset=torch.utils.data.TensorDataset(test_inputs, test_masks, test_labels),
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                                'attention_mask': torch.stack([item[1] for item in data]),
                                'labels': torch.stack([item[2] for item in data])},
    compute_metrics=lambda eval_prediction_output: compute_metrics(eval_prediction_output, test_labels)
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def predict(text):
    text = clean_text(text)
    encoded_review = tokenizer.encode_plus(text, max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True, #True,
    truncation='longest_first',
    return_attention_mask=True,
    return_tensors='pt'
    )
    input_ids = encoded_review['input_ids'].to(device) #(input_ids + ([tokenizer.pad_token_id] * padding_length)).to(device)  
    attention_mask = encoded_review['attention_mask'].to(device)
    
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output[0], dim=1)
    return prediction[0].cpu().detach().numpy()